In [115]:
# Cell 1: Function definition
def filter_tracks(df, min_track_length, id_label='id',frame_label='frame'):
    """
    Filter a pandas data frame to only include tracks that are at least min_track_length long.
    The data frame must contain a column with an id for each track (default: 'particle'), and a column representing timepoints (default: 'frame').
    """

    import trackpy as tp
    
    #relabel the column names to the labels used by trackpy
    df = df.rename(columns={id_label: 'particle'})
    df = df.rename(columns={frame_label: 'frame'})

    #filter the tracks
    tracks = tp.filter_stubs(df, min_track_length)
    #reset the index
    tracks = tracks.reset_index(drop=True)

    #revert the column names back to the original
    tracks = tracks.rename(columns={'particle': id_label})
    tracks = tracks.rename(columns={'frame': frame_label})

    return tracks

In [116]:
# Cell 2: Test function
def check(candidate):
    import pandas as pd

    ### test 1
    #create a test data frame
    df = pd.DataFrame({'id': [1, 1, 1, 2, 2, 3, 3, 3, 3], 'frame': [1, 2, 3, 1, 2, 1, 2, 3, 4]})

    #filter the data frame
    min_track_length = 3
    df_ground_truth = pd.DataFrame({'id': [1, 1, 1, 3, 3, 3, 3], 'frame': [1, 2, 3, 1, 2, 3, 4]})
    # --> track with id 2 is too short and should be removed

    df_filtered = candidate(df, min_track_length)
    #don't directly compare the data frames, as the index might be different and we don't care about it
    assert df_filtered.columns.equals(df_ground_truth.columns)
    assert (df_filtered.values == df_ground_truth.values).all()


    ### test 2
    #try with different min_track_length
    min_track_length = 1
    # --> all tracks should be kept
    df_filtered = candidate(df, min_track_length)
    assert df_filtered.columns.equals(df.columns)
    assert (df_filtered.values == df.values).all()


    ### test 3
    #try with a different id_label and frame_label
    df.rename(columns={'id': 'track_id', 'frame': 'time'}, inplace=True)
    df_ground_truth.rename(columns={'id': 'track_id', 'frame': 'time'}, inplace=True)
    df_filtered = candidate(df, 3, 'track_id', 'time')

    #check that there's no error when filtering out all tracks
    df_ground_truth_empty = pd.DataFrame({'id': [], 'frame': []})
    df.rename(columns={'track_id': 'id', 'time': 'frame'}, inplace=True)
    df_filtered = candidate(df, 9999999)
    assert df_filtered.columns.equals(df_ground_truth_empty.columns)
    assert (df_filtered.values == df_ground_truth_empty.values).all()


    ### test 4
    #test with a data frame that has no tracks
    df_filtered = candidate(df_ground_truth_empty, 3)
    assert df_filtered.columns.equals(df_ground_truth_empty.columns)
    assert (df_filtered.values == df_ground_truth_empty.values).all()



In [117]:
# Cell 3: Run test
check(filter_tracks)